## Identify genes in Oly transcriptome v3

The transcriptome that I used to pseudo-align counts in Salmon was created denovo from RNASeq data. That means that it is not tied to any of the feature files derived from the Oly genome. I therefore need to annotate the transcriptome, i.e. find genes. I can do this via blast. Luckily I have code from 2018 to do this! 

In this notebook I will blast the [denovo transcriptome (v3)](http://eagle.fish.washington.edu/cnidarian/Olurida_transcriptome_v3.fasta) fasta file, which is listed on the [Roberts Lab Genomic Resources page](https://github.com/RobertsLab/resources/wiki/Genomic-Resources) against Uniprot/Swissprot database to annotate.  

### Install Blast 

I downloaded the latest blast program files (ncbi-blast-2.10.0+-x64-macosx.tar.gz) from this [website](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/). I untarred, confirmed that the md5 matched, moved to my /Applications/bioinformatics/ directory and renamed to simply "blast/".  

In [2]:
# create path variable to blast directory 
blast = "/Applications/bioinformatics/blast/bin/"

In [4]:
# test blastx path variable 
! {blast}blastx -help

USAGE
  blastx [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-task task_name] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-negative_seqidlist filename]
    [-taxids taxids] [-negative_taxids taxids] [-taxidlist filename]
    [-negative_taxidlist filename] [-ipglist filename]
    [-negative_ipglist filename] [-entrez_query entrez_query]
    [-db_soft_mask filtering_algorithm] [-db_hard_mask filtering_algorithm]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]
    [-gapopen open_penalty] [-gapextend extend_penalty]
    [-qcov_hsp_perc float_value] [-max_hsps int_value]
    [-xdrop_ungap float_value] [-xdrop_gap float_value]
    [-xdrop_gap_final float_value] [-searchsp int_value]
    [-sum_stats bool_value] [-max_intron_length length] [-seg SEG_options]
    [-soft_masking soft_masking

### Create blast database using UniProt/Swiss-prot 

In [18]:
# create path variable to working directory, saved on my external hard drive 
workingdir = "/Volumes/Bumblebee/O.lurida_QuantSeq-2020/"

In [19]:
cd {workingdir}

/Volumes/Bumblebee/O.lurida_QuantSeq-2020


In [8]:
mkdir {workingdir}references/blastdb/

In [9]:
!curl \
ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz \
> {workingdir}references/blastdb/uniprot_sprot.fasta.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 85.1M  100 85.1M    0     0   449k      0  0:03:13  0:03:13 --:--:-- 1126k:05  0:02:56  519k1M    0     0   462k      0  0:03:08  0:01:24  0:01:44 1352k  0  0:03:00  0:01:40  0:01:20  615k 0:02:59  0:02:24  0:00:35  334k:03:00  0:02:25  0:00:35  351k


In [10]:
!gunzip -k {workingdir}references/blastdb/uniprot_sprot.fasta.gz

In [12]:
!{blast}makeblastdb \
-in {workingdir}references/blastdb/uniprot_sprot.fasta \
-dbtype prot \
-out {workingdir}references/blastdb/uniprot_sprot_20200513



Building a new DB, current time: 05/13/2020 17:04:02
New DB name:   /Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/blastdb/uniprot_sprot_20200513
New DB title:  /Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/blastdb/uniprot_sprot.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 562253 sequences in 14.3889 seconds.




In [14]:
! ls {workingdir}references/blastdb/

uniprot_sprot.fasta        uniprot_sprot_20200513.pot
uniprot_sprot.fasta.gz     uniprot_sprot_20200513.psq
uniprot_sprot_20200513.pdb uniprot_sprot_20200513.ptf
uniprot_sprot_20200513.phr uniprot_sprot_20200513.pto
uniprot_sprot_20200513.pin


## Run blast 

In [16]:
!{blast}blastx \
-query {workingdir}references/Olurida_transcriptome_v3.fasta \
-db {workingdir}references/blastdb/uniprot_sprot_20200513 \
-out {workingdir}references/Olurida_transcriptome_v3_blastx.tab \
-evalue 1E-20 \
-num_threads 8 \
-outfmt 6

### Look at blast results 

In [3]:
! head {workingdir}references/Olurida_transcriptome_v3_blastx.tab

comp75_c0_seq1	sp|Q3SWT6|PPE1_RAT	69.737	76	23	0	229	2	342	417	1.65e-35	129
comp75_c0_seq1	sp|O35385|PPE2_MOUSE	71.053	76	22	0	229	2	441	516	5.55e-35	127
comp75_c0_seq1	sp|O14830|PPE2_HUMAN	68.421	76	24	0	229	2	437	512	1.86e-34	126
comp75_c0_seq1	sp|O14829|PPE1_HUMAN	72.368	76	21	0	229	2	352	427	1.64e-33	123
comp75_c0_seq1	sp|O35655|PPE1_MOUSE	67.532	77	25	0	232	2	352	428	2.73e-33	123
comp75_c0_seq1	sp|P40421|RDGC_DROME	61.842	76	29	0	229	2	309	384	3.05e-31	117
comp75_c0_seq1	sp|G5EBX9|PPE_CAEEL	63.158	76	28	0	229	2	414	489	4.84e-31	116
comp147_c0_seq1	sp|Q8IWF2|FXRD2_HUMAN	59.821	112	44	1	5	337	187	298	2.74e-39	141
comp147_c0_seq1	sp|Q3USW5|FXRD2_MOUSE	57.143	112	47	1	5	337	178	289	3.45e-37	135
comp147_c0_seq1	sp|B0UXS1|FXRD2_DANRE	50.943	106	52	0	5	322	187	292	7.07e-31	118


In [4]:
# 2,518,129 hits 
! wc -l {workingdir}references/Olurida_transcriptome_v3_blastx.tab

 2518129 /Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/Olurida_transcriptome_v3_blastx.tab


### Convert blast output from tabular to GFF format 

I need a GFF/GTF formatted annotation file to use in Tximport, which is the program that processes Salmon transcript quantification data into gene counts.  I found a function in the program MGKit, called `blast2gff`, which does just that. To use, I need to install MGKit. Did so via Bioconda according to the manual's instructions: 

### Down a conda environment rabbit hole: 
Based on my experience/trouble shooting, programs that are installed via conda in their own environments are not accessible via the terminal until environments are "activated". To do so, just type `conda activate <program>` in the terminal, then the package commands should be accessible. 

However, this doesn't work in Jupyter Notebooks automatically. Following [this blog post](https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084) I set up a conda environment called "env_bioinf", installed the packages that I need to access, and then added that as a kernel to my Jupyter notebook. Here is the full list of commands I used. Note: this was all entered in my terminal (i.e. NOT in Jupyter notebook:

    conda create --name env_bioinf
    conda activate env_bioinf
    conda install -c conda-forge mgkit
    conda install -c conda-forge salmon #I need that for another part of my pipeline, so installed that in the env_bioinf env too 
    conda install -c anaconda ipykernel
    python -m ipykernel install --user --name=env_bioinf


I then closed my Jupyter Notebook, re-opened, and voila, I have a new kernel called "env_bioinf".  

![image](https://user-images.githubusercontent.com/17264765/82086294-28689680-96a3-11ea-87f9-a5115d12d767.png)

#### Helpful tip:  

To view all conda environments, type `conda env list` in terminal. When I do that, I get: 

In [10]:
! conda env list

# conda environments:
#
base                  *  /Users/laura/anaconda3
env_bioinf               /Users/laura/anaconda3/envs/env_bioinf
salmon                   /Users/laura/anaconda3/envs/salmon



In [11]:
! salmon --version

salmon 1.2.1


In [12]:
! blast2gff --version

/bin/sh: blast2gff: command not found


For some reason I still cannot access my mgkit functions.  Let's try to access it directoy: 

In [4]:
# Explore anaconda3/ subdirectories 
! ls /Users/laura/anaconda3/envs/env_bioinf

bin        include    man        share
conda-meta lib        sbin       ssl


In [5]:
# blast2gff is in there 
! ls /Users/laura/anaconda3/envs/env_bioinf/bin/

2to3                     h5perf_serial            pasteurize
2to3-3.7                 h5redeploy               pip
add-gff-info             h5repack                 pkgdata
blast2gff                h5repart                 png-fix-itxt
bunzip2                  h5stat                   pngfix
bzcat                    h5unjam                  pt2to3
bzcmp                    h5watch                  ptdump
bzdiff                   hmmer2gff                ptrepack
bzegrep                  htseq-count              pttree
bzfgrep                  htseq-count-barcodes     pydoc
bzgrep                   htseq-qa                 pydoc3
bzip2                    icu-config               pydoc3.7
bzip2recover             icuinfo                  pygmentize
bzless                   idle3                    python
bzmore                   idle3.7                  python3
c_rehash                 infocmp                  python3-config
captoinfo                infotocap                python3.7
char

In [7]:
# I can also access blast2gff via hard-coding 
! /Users/laura/anaconda3/envs/env_bioinf/bin/blast2gff --version

blast2gff, version 0.4.3


### Create a path variable to my env_bioinf environment to access packages 

In [15]:
env_bioinf = "/Users/laura/anaconda3/envs/env_bioinf/bin/"

In [16]:
! {env_bioinf}blast2gff --version

blast2gff, version 0.4.3


### Convert tabular blast results to GFF 

In [24]:
! {env_bioinf}blast2gff \
uniprot \
--progress \
{workingdir}references/Olurida_transcriptome_v3_blastx.tab \
{workingdir}references/Olurida_transcriptome_v3_blastx.gff

INFO - mgkit.workflow.blast2gff: Writing to file (/Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/Olurida_transcriptome_v3_blastx.gff)
0it [00:00, ?it/s]INFO - mgkit.io.blast: Reading BLAST results from file (/Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/Olurida_transcriptome_v3_blastx.tab)
2518106it [01:47, 23752.18it/s]INFO - mgkit.io.blast: Read 2518129 BLAST records
2518129it [01:47, 23504.20it/s]


In [27]:
# original tabular blast results 
! head -3 {workingdir}references/Olurida_transcriptome_v3_blastx.tab

comp75_c0_seq1	sp|Q3SWT6|PPE1_RAT	69.737	76	23	0	229	2	342	417	1.65e-35	129
comp75_c0_seq1	sp|O35385|PPE2_MOUSE	71.053	76	22	0	229	2	441	516	5.55e-35	127
comp75_c0_seq1	sp|O14830|PPE2_HUMAN	68.421	76	24	0	229	2	437	512	1.86e-34	126


In [28]:
# format after conversion to GFF 
! head -3 {workingdir}references/Olurida_transcriptome_v3_blastx.gff

comp75_c0_seq1	BLAST	CDS	2	229	129.0	-	0	bitscore="129.0";db="UNIPROT-SP";dbq="10";evalue="1.65e-35";frame="r0";gene_id="Q3SWT6";identity="69.737";subject_end="417";subject_start="342";uid="d6d01f18-2217-4e2e-93a5-2e67fccde5e6"
comp75_c0_seq1	BLAST	CDS	2	229	127.0	-	0	bitscore="127.0";db="UNIPROT-SP";dbq="10";evalue="5.55e-35";frame="r0";gene_id="O35385";identity="71.053";subject_end="516";subject_start="441";uid="680a3ef7-2b25-475f-9c55-582e2675fec9"
comp75_c0_seq1	BLAST	CDS	2	229	126.0	-	0	bitscore="126.0";db="UNIPROT-SP";dbq="10";evalue="1.86e-34";frame="r0";gene_id="O14830";identity="68.421";subject_end="512";subject_start="437";uid="18f2dc4b-109c-40bc-815e-e603ba9a1a52"


In [29]:
! wc -l {workingdir}references/Olurida_transcriptome_v3_blastx.gff

 2518129 /Volumes/Bumblebee/O.lurida_QuantSeq-2020/references/Olurida_transcriptome_v3_blastx.gff


In [30]:
! echo $PATH

/usr/local/opt/icu4c/sbin:/usr/local/opt/icu4c/bin:/Users/laura/anaconda3/bin:/Users/laura/anaconda3/condabin:/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin


### The blast results in GFF format is now ready to be used with Tximport in R to convert trancript counts to gene counts

..... Hold that thought. 

I don't know why, but the  script in Tximport's vignette to create a tx2gene object doesn't work for me. _However_, all I need is a file with two columns: 1) transcript ID and 2) gene ID, like this: 

![image](https://user-images.githubusercontent.com/17264765/82094687-67eaaf00-96b2-11ea-9883-869b32751c4e.png)

I can create this from my blast tabular results from the tab file, which I will do in R.  Also, importantly I have multiple gene hits per transcript, so I will extract the "top hit gene" (also in R).   

Pro Tip - Sam recommends using Diamond Blast 